# <span style="color:#84b6f4;">Detección de comentarios de hate</span>

### <span style="color:#77dd77;">Formación del grafo</span>

En primer lugar, sacamos los datos del fichero txt y formamos un DataFrame con ellos:

In [1]:
import pandas as pd
import os

# Especificamos el archivo de texto
txt_file = os.getcwd()+'/resources/labeled_corpus_6K.txt'

# Leemos el archivo de texto con el formato específico
data = []
with open(txt_file, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split(';||;')
        if len(parts) == 3:
            data.append(parts)

# Creamos un DataFrame a partir de los datos
column_names = ['id', 'text', 'hate']
data_frame = pd.DataFrame(data, columns=column_names)

# Convertimos la columna 'hate' en enteros
data_frame['hate'] = data_frame['hate'].astype(int)

# Mostramos las primeras filas del DataFrame
print(data_frame.head(5))

                      id                                               text   
0  id=828025263321657348  Ismael es egocentrico porque se vuelve loca si...  \
1  id=828025128797741057  ..ya tardaba en salir quien pronunciase nombre...   
2  id=828025087815274496  (Esto no es un discurso político y razonado, o...   
3  id=828025006626058241  Muy despreciados,siiii,pero todos vestidos de ...   
4  id=828024709761658880  marica explicame porque a veces no te entiendo...   

   hate  
0     0  
1     0  
2     0  
3     1  
4     1  


### <span style="color:#77dd77;">Entrenamiento</span>

Seleccionamos las métricas que usaremos para predecir, así como el atributo objetivo que, en este caso, será numérico.

In [2]:
# Atributo que usaremos para predecir
attributes = data_frame[['text']]

# Atributo objetivo a predecir
goal = data_frame['hate']

A continuación, entrenamos un modelo de clasificación con una red neuronal:

In [109]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

text = data_frame['text']

# Dividimos los datos en conjuntos de entrenamiento (70%) y prueba (30%)
attributes_train, attributes_test, goal_train, goal_test = train_test_split(text, goal, test_size=0.3, random_state=42)

# Creamos un vectorizador de texto
vectorizer = CountVectorizer()
tweet_train_vectorized = vectorizer.fit_transform(attributes_train)
tweet_test_vectorized = vectorizer.transform(attributes_test)

# Entrenamos el modelo de clasificación de redes neuronales
mlp_classifier = MLPClassifier(hidden_layer_sizes=(50, 50, 50), activation='relu', solver='adam', random_state=42, max_iter=1000)
mlp_classifier.fit(tweet_train_vectorized, goal_train)

# Realizamos predicciones con el conjunto de prueba
prediction = mlp_classifier.predict(tweet_test_vectorized)

# Calculamos la precisión del modelo
accuracy = accuracy_score(goal_test, prediction)
print("Precisión del modelo en el conjunto de prueba:", accuracy)

# Mostramos la matriz de confusión
print(confusion_matrix(goal_test, prediction))

Precisión del modelo en el conjunto de prueba: 0.7744444444444445
[[1176  157]
 [ 249  218]]


Para finalizar, probamos el modelo con algunos ejemplos concretos:

In [129]:
message = ["respetuoso", "de hate"]

# Texto de ejemplo 1
example_text_1 = "No estoy de acuerdo con tu análisis, creo que la película es muy entretenida."
example_text_1_vectorized = vectorizer.transform([example_text_1])

# Texto de ejemplo 2
example_text_2 = "Hay que ser idiota para hacer esta reseña."
example_text_2_vectorized = vectorizer.transform([example_text_2])

# Texto de ejemplo 3
example_text_3 = "¡Enhorabuena por tu trabajo!"
example_text_3_vectorized = vectorizer.transform([example_text_3])

# Texto de ejemplo 4
example_text_4 = "Tu trabajo es basura y tú también."
example_text_4_vectorized = vectorizer.transform([example_text_4])

print("El comentario '" + example_text_1 + "' es " + message[mlp_classifier.predict(example_text_1_vectorized)[0]] + ".")
print("El comentario '" + example_text_2 + "' es " + message[mlp_classifier.predict(example_text_2_vectorized)[0]] + ".")
print("El comentario '" + example_text_3 + "' es " + message[mlp_classifier.predict(example_text_3_vectorized)[0]] + ".")
print("El comentario '" + example_text_4 + "' es " + message[mlp_classifier.predict(example_text_4_vectorized)[0]] + ".")

El comentario 'No estoy de acuerdo con tu análisis, creo que la película es muy entretenida.' es respetuoso.
El comentario 'Hay que ser idiota para hacer esta reseña.' es de hate.
El comentario '¡Enhorabuena por tu trabajo!' es respetuoso.
El comentario 'Tu trabajo es basura y tú también.' es de hate.
